In [4]:
def find_true_indices(lst, N):
    true_indices = [i for i, val in enumerate(lst) if val]
    return true_indices[:N]


import re

def replace_multiple_newlines(text):
    # 3つ以上の連続する改行を2つの改行に置換
    return re.sub(r'\n{2,}', '\n\n', remove_spaces_from_blank_lines(text))

def remove_spaces_from_blank_lines(text):
    # 各行を処理し、スペースのみの行を空行に置き換える
    return re.sub(r'^\s+$', '', text, flags=re.MULTILINE)

def select_usable_ymm(usable_var):
    count_true = sum(usable_var)
    count_mod2 = (count_true>>1)<<1
    count_half = count_true>>1
    indices = find_true_indices(usable_var, count_mod2)
    
    vars1 = [f"ymm{idx}" for idx in indices[:count_half]]
    vars2 = [f"ymm{idx}" for idx in indices[count_half:]]
    for idx in indices:
        usable_var[idx] = False
    return vars1, vars2, usable_var 

def find_true_indices(lst, N):
    true_indices = [i for i, val in enumerate(lst) if val]
    return true_indices[:N]

def select_usable_var(usable_var, prefix="r", set_second=False):
    var1, var2 = "", ""
    indices = find_true_indices(usable_var, 2)
    
    var1 = f"{prefix}{indices[0]}"
    var2 = f"{prefix}{indices[1]}"
    usable_var[indices[0]] = False
    if set_second: usable_var[indices[1]] = False
    return var1, var2, usable_var

def accumulator_simd(accm_op, variables, indent_level=1):
    indent = " " * 4 * indent_level
    
    accm_op.append("\n")
    if len(variables) == 1:
        accm_op.append(variables[0])
        return 

    variables_next_ = []
    variables_ = variables[:-1] if len(variables) % 2 == 1 else variables
    for i in range(0, len(variables_), 2):
        accm_op.append(f"{indent}{variables_[i]} = _mm256_add_pd({variables_[i]}, {variables_[i+1]});")
        variables_next_.append(f"{variables_[i]}")
        
    if len(variables) % 2 == 1:
        variables_next_.append(f"{variables[-1]}")
        
    accumulator_simd(accm_op, variables_next_, indent_level = indent_level)

def accumulator_naive_c(accm_op, variables, indent_level=1):
    indent = " " * 4 * indent_level
    
    accm_op.append("\n")
    if len(variables) == 1:
        accm_op.append(variables[0])
        return 

    variables_next_ = []
    variables_ = variables[:-1] if len(variables) % 2 == 1 else variables
    for i in range(0, len(variables_), 2):
        accm_op.append(f"{indent}{variables_[i]} = {variables_[i]} + {variables_[i+1]};")
        variables_next_.append(f"{variables_[i]}")
        
    if len(variables) % 2 == 1:
        variables_next_.append(f"{variables[-1]}")
        
    accumulator_naive_c(accm_op, variables_next_, indent_level = indent_level)


with open(f"./small_dgemv_naive_c_implementation_ver2_4.c", "w") as p:
    p.write("#include <stddef.h>\n")
    p.write("#include <stdio.h>\n")
    p.write("#include <stdlib.h>\n")
    p.write("#include <stdint.h>\n")
    p.write("#include <math.h>\n")
    p.write("#include <omp.h>\n")
    p.write("#include <immintrin.h>\n") 
    p.write("""s
double sum_ymm_elements_v2_4(__m256d ymmX) {
    __m128d ymm_upper = _mm256_extractf128_pd(ymmX, 1);
    __m128d ymm_lower = _mm256_extractf128_pd(ymmX, 0);

    __m128d ymm_sum = _mm_add_pd(ymm_upper, ymm_lower);

    __m128d ymm_high_low = _mm_add_pd(ymm_sum, _mm_unpackhi_pd(ymm_sum, ymm_sum));

    double result;
    _mm_store_sd(&result, ymm_high_low);

    return result;
}    
    """)

    for ldx in range(4, 516, 4):
        ldx_max = ldx // 2
        declare_ymm = "__m256d " + ", ".join([f"ymm{ld}" for ld in range(ldx_max)])
        variables = [f"ymm{ld}" for ld in range(ldx_max)]

        usable_var = [True] * ldx_max
        
        vars1, vars2, usable_var = select_usable_ymm(usable_var)
    
        load_ymm = ""
        op_concat = ""

        
        for i, var in enumerate(vars2):
            load_ymm += f"""
        {var} = _mm256_loadu_pd(&x[{i*4}]);"""
        
        if ldx<=1000:
            if ldx <=16:
                for i, var in enumerate(vars1):
                    op_concat += f"""
            {var} = _mm256_loadu_pd(&a_t[{i*4}]);"""

                op_concat += "\n"
                for var1, var2 in zip(vars1, vars2):
                    op_concat += f"""
            {var1} = _mm256_mul_pd({var1}, {var2});"""

                accm_variables = vars1
                accm_op = []
                accumulator_naive_c(accm_op, accm_variables, indent_level=2)
                op_concat += "\n".join(accm_op[:-1])

                op_concat += f"\n        y[i] = sum_ymm_elements_v2_4({accm_op[-1]}); a_t += {ldx};"

            else:
                for i, var in enumerate(vars1):
                    op_concat += f"""
            {var} = _mm256_loadu_pd(&a_t[{i*4}]);"""
                    
                for var1, var2 in zip(vars1, vars2):
                    op_concat += f"""
            {var1} = _mm256_mul_pd({var1}, {var2});"""

                accm_variables = vars1
                accm_op = []
                accumulator_naive_c(accm_op, accm_variables, indent_level=2)
                op_concat += "\n".join(accm_op[:-1])
                for var1, var2 in zip(vars1, vars2):
                    usable_var[int(var1[3:])] = True
                    usable_var[int(var2[3:])] = True
                usable_var[0] = False
                op_concat += f"\n        y[i] = sum_ymm_elements_v2_4({accm_op[-1]}); a_t += {ldx};"
            
        base = f"""
void mydgemv_t_ver2_4_{ldx}(double a_t[], double x[], double y[], int64_t lda, int64_t ldx, int64_t ldy){{
    {declare_ymm};
    {load_ymm}
    
    for (int i=0; i<lda; i++){{
        {op_concat}
        
    }}
}}
        """
        p.write(base)
        
        
        ldx_max = ldx * 2 + 1
        declare_tmp_x = "double " + ", ".join([f"tmp_x_{ld} = x[{ld}]" for ld in range(ldx_max)])
        declare_xmm = "__m256d " + ", ".join([f"ymm{ld}" for ld in range(ldx_max)])
        variables = [f"ymm{ld}" for ld in range(ldx_max)]

        usable_var = [True] * ldx_max

        vars1, vars2, usable_var = select_usable_ymm(usable_var)

        load_a = ""
        load_xmm = ""
        comp_remain_ax = ""
        op_concat = ""

        # -------------------------------------------------------------------------------------------------
        # Load X
        for k, var in enumerate(vars2):
            load_xmm += f"""{var} = _mm256_set1_pd(x[{k}]);
        """

        # Load A
        for k, var in enumerate(vars1):
            load_a += f"""{var} = _mm256_loadu_pd(&a[i+{k}*lda]);
        """

        # multipy A x X
        mul_op = ""
        for var1, var2 in zip(vars1, vars2):
            mul_op += f"""{var1} = _mm256_mul_pd({var2}, {var1});
        """
        # Accumulate AxX to y
        accm_variables = [f"ymm{ldx_max-1}"] + vars1
        accm_op = []
        accumulator_simd(accm_op, accm_variables, indent_level=2)
        accm_op = "\n".join(accm_op[:-1])

        # -------------------------------------------------------------------------------------------------        
        # Compute Remain 
        for k in range(ldx_max//2):
            comp_remain_ax += f"""tmp_a_{k} = a[i+{k}*lda] * x[{k}];
            """
        # Accumulate AxX to y
        accm_variables = [f"tmp_a_{k}" for k in range(ldx_max//2)] 
        accm_remain_op = []
        accumulator_naive_c(accm_remain_op, accm_variables, indent_level=2)
        accm_remain_op = "\n".join(accm_remain_op[1:-1])

        base = f"""
void mydgemv_n_ver2_4_{ldx}(double a[], double x[], double y[], int64_t lda, int64_t ldx, int64_t ldy){{
    {declare_xmm};
    {load_xmm}

    // Init y
    for (int64_t i=0; i<lda; i++){{
        y[i] = 0.0;
    }}

    int64_t i_remain = lda % 4;
    for (int64_t i=0; i<lda-i_remain; i+=4){{
        // Load y
        ymm{ldx_max-1} = _mm256_loadu_pd(&y[i]);

        // Load a
        {load_a}

        // Multiply
        {mul_op}

        // Accumulate
        {accm_op}

        // Store y
        _mm256_storeu_pd(&y[i], ymm{ldx_max-1});
    }}

    double tmp_y;
    double {", ".join([f"tmp_a_{i}" for i in range(ldx_max//2)])};
    for (int64_t i=lda-i_remain; i<lda; i++){{
        // Load y
        tmp_y = y[i];

        // Compute AxX
        {comp_remain_ax}

        // Accumulate AxX
{accm_remain_op}

        // Store y
        y[i] = tmp_a_0;
    }}
}}
            """
        p.write(replace_multiple_newlines(base))        

In [6]:
for i in range(4, 516, 4):
    print(f"""
        subroutine mydgemv_n_ver2_4_{i}(a, x, y, lda, ldx, ldy) bind(C, name='mydgemv_n_ver2_4_{i}')
            import
            integer(c_int64_t), value :: lda, ldx, ldy
            real(c_double), intent(in) :: a(lda, ldx), x(ldx), y(ldy)
        end subroutine mydgemv_n_ver2_4_{i}
          """)
#     print(f"if (ldx == {i}) sub_ptr => mydgemv_t_ver2_4_{i}")


        subroutine mydgemv_n_ver2_4_4(a, x, y, lda, ldx, ldy) bind(C, name='mydgemv_n_ver2_4_4')
            import
            integer(c_int64_t), value :: lda, ldx, ldy
            real(c_double), intent(in) :: a(lda, ldx), x(ldx), y(ldy)
        end subroutine mydgemv_n_ver2_4_4
          

        subroutine mydgemv_n_ver2_4_8(a, x, y, lda, ldx, ldy) bind(C, name='mydgemv_n_ver2_4_8')
            import
            integer(c_int64_t), value :: lda, ldx, ldy
            real(c_double), intent(in) :: a(lda, ldx), x(ldx), y(ldy)
        end subroutine mydgemv_n_ver2_4_8
          

        subroutine mydgemv_n_ver2_4_12(a, x, y, lda, ldx, ldy) bind(C, name='mydgemv_n_ver2_4_12')
            import
            integer(c_int64_t), value :: lda, ldx, ldy
            real(c_double), intent(in) :: a(lda, ldx), x(ldx), y(ldy)
        end subroutine mydgemv_n_ver2_4_12
          

        subroutine mydgemv_n_ver2_4_16(a, x, y, lda, ldx, ldy) bind(C, name='mydgemv_n_ver2_4_16')
            